<b>Equal-Weight S&P 500 Index Fund</b>

Introduction & Library Imports
The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [3]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
stocks=pd.read_csv("sp_500_stocks.csv")

In [3]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [2]:
#Secets.py file
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [4]:
base_url="https://sandbox.iexapis.com"
symbol="AAPL"
'''
You add token inorder to access the end-point
'''
api_url=f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"

In [5]:
print(api_url)

https://sandbox.iexapis.com/stable/stock/AAPL/quote/?token=Tpk_059b97af715d417d9f49f50b51b1c448


In [10]:
#Running the GET request
data=requests.get(api_url)

In [11]:
#Up and Running
data.status_code

200

In [12]:
'''Transform the Data into .json format inorder to access the data'''
data=requests.get(api_url).json()

In [13]:
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'TG AEARGLQ)A LOKSS NDEMTECBNL(/SA', 'calculationPrice': 'tops', 'open': 0, 'openTime': None, 'openSource': 'lifiocfa', 'close': 0, 'closeTime': None, 'closeSource': 'cfiliaof', 'high': 0, 'highTime': None, 'highSource': None, 'low': 0, 'lowTime': None, 'lowSource': None, 'latestPrice': 151.99, 'latestSource': 'IEX real time price', 'latestTime': '1:27:55 PM', 'latestUpdate': 1630831256541, 'latestVolume': None, 'iexRealtimePrice': 147.78, 'iexRealtimeSize': 5, 'iexLastUpdated': 1696784281776, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 151.2, 'previousVolume': 78550991, 'change': 1.85, 'changePercent': 0.01293, 'volume': None, 'iexMarketPercent': 0.012914461376442766, 'iexVolume': 814745, 'avgTotalVolume': 84230575, 'iexBidPrice': 153.23

In [14]:
#Getting the Market Cap and the Latest Price
price=data["latestPrice"]
market_cap=data["marketCap"]

In [15]:
price

151.99

In [16]:
market_cap

2453620955247

In [17]:
my_columns=["Ticker" ,"Stock_Price","Market_Captilization","Num_Shares_to_Buy"]
final_dataframe=pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock_Price,Market_Captilization,Num_Shares_to_Buy


In [18]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
        
    ],index=my_columns
    ),ignore_index=True
    
)

,Ticker,Stock_Price,Market_Captilization,Num_Shares_to_Buy
0,AAPL,151.99,2453620955247,N/A


In [ ]:
final_dataframe=pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    api_url=f"https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}"
    data=requests.get(api_url).json()
    final_dataframe=final_dataframe.append(
    pd.Series([
        stock,
        data["latestPrice"],
        data["marketCap"],
        'N/A'
    ],index=my_columns
    ),ignore_index=True
    )
    

In [ ]:
final_dataframe


<b>Using Batch API Calls to Improve Performance</b>

Batch API calls are one of the easiest ways to improve the performance of your code.


This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [19]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [20]:
symbol_groups=list(chunks(stocks['Ticker'],100))

In [32]:
symbol_groups

[0         A
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
       ...  
 95     CINF
 96       CL
 97      CLX
 98      CMA
 99    CMCSA
 Name: Ticker, Length: 100, dtype: object,
 100     CME
 101     CMG
 102     CMI
 103     CMS
 104     CNC
        ... 
 195    FTNT
 196     FTV
 197      GD
 198      GE
 199    GILD
 Name: Ticker, Length: 100, dtype: object,
 200     GIS
 201      GL
 202     GLW
 203      GM
 204    GOOG
        ... 
 295     MAA
 296     MAR
 297     MAS
 298     MCD
 299    MCHP
 Name: Ticker, Length: 100, dtype: object,
 300     MCK
 301     MCO
 302    MDLZ
 303     MDT
 304     MET
        ... 
 395     RHI
 396     RJF
 397      RL
 398     RMD
 399     ROK
 Name: Ticker, Length: 100, dtype: object,
 400     ROL
 401     ROP
 402    ROST
 403     RSG
 404     RTX
        ... 
 495    XLNX
 496     XOM
 497    XRAY
 498     XRX
 499     XYL
 Name: Ticker, Length: 100, dtype: object,
 500     YUM
 501     ZBH
 502    ZBRA
 503    ZION
 504     ZTS
 Name

In [23]:
len(symbol_groups)

6

In [34]:

symbol_strings=[]
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))


6
A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,I

In [40]:
batch_api_call=f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_strings[0]}&types=quote&token={IEX_CLOUD_API_TOKEN}'
print(batch_api_call)

https://sandbox.iexapis.com/stable/stock/market/batch?symbols=A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA&types=quote&token=Tpk_059b97af715d417d9f49f50b51b1c448


In [43]:
final_dataframe=pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call=f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data =requests.get(batch_api_call).json()
    for symbol in symbol_string.split(","):
        final_dataframe=final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]["quote"]["latestPrice"],
                data[symbol]["quote"]["marketCap"],
                'N/A'
                
            ],index=my_columns
            ),ignore_index=True
        )

In [44]:
final_dataframe

,Ticker,Stock_Price,Market_Captilization,Num_Shares_to_Buy
0,A,156.210,45565180913,N/A
1,AAL,20.840,13449491139,N/A
2,AAP,216.720,14123715139,N/A
3,AAPL,148.770,2476087145279,N/A
4,ABBV,120.900,211663188906,N/A
...,...,...,...,...
500,YUM,120.179,35043512799,N/A
501,ZBH,159.960,33424208194,N/A
502,ZBRA,539.919,29489914192,N/A
503,ZION,53.760,8738312395,N/A


In [38]:
batch_api_call=f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"

In [39]:
print(batch_api_call)

https://sandbox.iexapis.com/stable/stock/market/batch?symbols=YUM,ZBH,ZBRA,ZION,ZTS&types=quote&token=Tpk_059b97af715d417d9f49f50b51b1c448


In [49]:
val=input("Enter the value of Portfolio :")
try :
    portfolio_size=float(val)
except ValueError:
    portfolio_size=float(input("Please enter a Number :"))

Enter the value of Portfolio :100000


Position Size is the amount of money who will invest in per size

In [50]:

position_size=portfolio_size/len(final_dataframe)
for i in range(0,len(final_dataframe)):
    #Find the nummber of shares we are gonna buy for each stock
    
    
    final_dataframe.loc[i,"Num_Shares_to_Buy"]=math.floor(position_size/final_dataframe.loc[i,"Stock_Price"])
final_dataframe

,Ticker,Stock_Price,Market_Captilization,Num_Shares_to_Buy
0,A,156.210,45565180913,1
1,AAL,20.840,13449491139,9
2,AAP,216.720,14123715139,0
3,AAPL,148.770,2476087145279,1
4,ABBV,120.900,211663188906,1
...,...,...,...,...
500,YUM,120.179,35043512799,1
501,ZBH,159.960,33424208194,1
502,ZBRA,539.919,29489914192,0
503,ZION,53.760,8738312395,3


In [51]:
position_size

198.01980198019803

In [29]:
len(final_dataframe.index)

505

In [77]:
writer=pd.ExcelWriter("recommended_trade.xlsx",engine = "xlsxwriter")
final_dataframe.to_excel(writer, "Recommended Trades",index = False)

In [78]:
background_color="#0a0a23"
font_color="#CCEEFF"

string_format=writer.book.add_format(
    {
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)

dollar_format=writer.book.add_format(
    {
        'num_format':"$0.00",
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)

integer_format=writer.book.add_format(
    {
        'num_format':"0",
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)

In [56]:
writer.sheets['Recommended Trades'].set_column('A:A',18, string_format)
writer.save()

In [63]:
column_formats['A'][1]

['Ticker', <xlsxwriter.format.Format at 0x18b1c5c49d0>]

In [79]:
column_formats={
    'A':['Ticker',string_format],
    'B':['Stock_Price',dollar_format],
    'C':['Market_Captilization',dollar_format],
    'D':['Num_Shares_to_Buy',integer_format]
}

for col in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{col}:{col}',width=25 ,cell_format=column_formats[col][1])
writer.save()

In [65]:
final_dataframe

,Ticker,Stock_Price,Market_Captilization,Num_Shares_to_Buy
0,A,156.210,45565180913,1
1,AAL,20.840,13449491139,9
2,AAP,216.720,14123715139,0
3,AAPL,148.770,2476087145279,1
4,ABBV,120.900,211663188906,1
...,...,...,...,...
500,YUM,120.179,35043512799,1
501,ZBH,159.960,33424208194,1
502,ZBRA,539.919,29489914192,0
503,ZION,53.760,8738312395,3
